# 01. Setup & Validation

## AI 기반 저탄소 시멘트 대체재 발견 파이프라인

**목적**: 환경 검증 및 파이프라인 모듈 테스트

---

### 체크리스트
- [ ] Python 환경 확인
- [ ] GPU (CUDA) 확인
- [ ] 필수 패키지 확인
- [ ] src/ 모듈 import 테스트
- [ ] CHGNet 동작 테스트
- [ ] 설정 파일 로딩 테스트

## 1. 환경 설정

In [1]:
# 프로젝트 루트를 Python path에 추가
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project Root: {PROJECT_ROOT}")
print(f"Python Path: {sys.path[0]}")

Project Root: c:\cement_final
Python Path: c:\cement_final


In [2]:
# 기본 라이브러리 확인
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import json

print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print("Matplotlib: OK")
print("YAML: OK")
print("JSON: OK")

NumPy: 2.4.1
Pandas: 3.0.0
Matplotlib: OK
YAML: OK
JSON: OK


## 2. GPU 확인

In [3]:
import torch

print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("\n⚠️ WARNING: GPU not available. Simulations will be slow.")

PyTorch: 2.7.1+cu118
CUDA Available: True
CUDA Version: 11.8
GPU: NVIDIA GeForce RTX 4070 Laptop GPU
GPU Memory: 8.6 GB


## 3. 시뮬레이션 패키지 확인

In [4]:
# ASE (Atomic Simulation Environment)
import ase
from ase import Atoms
from ase.io import read, write
from ase.build import molecule

print(f"ASE: {ase.__version__}")

# CHGNet
from chgnet.model import CHGNetCalculator
print("CHGNet: OK")

print("\n✓ All simulation packages available")

ASE: 3.27.0
CHGNet: OK

✓ All simulation packages available


## 4. src/ 모듈 Import 테스트

In [5]:
# 메인 모듈
import src
print(f"src version: {src.__version__}")
print(f"Project Root: {src.PROJECT_ROOT}")
print(f"Config Dir: {src.CONFIG_DIR}")

src version: 1.0.0
Project Root: c:\cement_final
Config Dir: c:\cement_final\config


In [6]:
# Core 모듈
from src.core import get_calculator, optimize_structure
from src.core import run_md_simulation
from src.core import create_hydration_system, add_water_molecules

print("✓ src.core modules loaded")

✓ src.core modules loaded


In [7]:
# Analysis 모듈
from src.analysis import (
    analyze_ca_leaching,
    analyze_si_coordination,
    analyze_csh_formation,
    calculate_screening_score
)

print("✓ src.analysis modules loaded")

✓ src.analysis modules loaded


In [8]:
# Database 모듈
from src.database import CandidateDatabase, Candidate

print("✓ src.database modules loaded")

✓ src.database modules loaded


In [9]:
# Pipeline 모듈
from src.pipeline import ScreeningPipeline, load_config

print("✓ src.pipeline modules loaded")

✓ src.pipeline modules loaded


In [10]:
# Visualization 모듈
from src.visualization import plot_screening_comparison, plot_evolution

print("✓ src.visualization modules loaded")
print("\n" + "="*50)
print("✓ ALL MODULES LOADED SUCCESSFULLY")
print("="*50)

✓ src.visualization modules loaded

✓ ALL MODULES LOADED SUCCESSFULLY


## 5. 설정 파일 로딩 테스트

In [11]:
# 시뮬레이션 설정 로딩
sim_config = load_config("simulation")

print("Simulation Config:")
print(f"  Baseline: {sim_config['baseline']['material']}")
print(f"  Optimization Fmax: {sim_config['optimization']['fmax']} eV/Å")
print(f"  MD Duration (screening): {sim_config['md_simulation']['screening']['duration_ps']} ps")
print(f"  MD Duration (deep): {sim_config['md_simulation']['deep_analysis']['duration_ps']} ps")

Simulation Config:
  Baseline: C3S
  Optimization Fmax: 0.05 eV/Å
  MD Duration (screening): 10 ps
  MD Duration (deep): 100 ps


In [12]:
# 평가 설정 로딩
eval_config = load_config("evaluation")

print("Evaluation Config:")
print(f"  Weights: {eval_config['scoring']['weights']}")
print(f"  Grades: {eval_config['grades']}")
print(f"  Baseline Ca Rate: {eval_config['baseline']['ca_leaching_rate']} Ca/ps")

Evaluation Config:
  Weights: {'co2_reduction': 0.3, 'ca_activity': 0.2, 'si_stability': 0.25, 'csh_formation': 0.25}
  Grades: {'A': 70, 'B': 50, 'C': 30, 'D': 0}
  Baseline Ca Rate: 0.1 Ca/ps


## 6. CHGNet 동작 테스트

In [13]:
# 간단한 구조로 CHGNet 테스트
print("Testing CHGNet Calculator...")

# 물 분자 생성
water = molecule('H2O')
water.set_cell([10, 10, 10])
water.set_pbc(True)

# 계산기 설정
calc = get_calculator(use_gpu=torch.cuda.is_available())
water.calc = calc

# 에너지 계산
energy = water.get_potential_energy()
forces = water.get_forces()

print(f"\n✓ CHGNet Test Passed")
print(f"  Water Energy: {energy:.4f} eV")
print(f"  Max Force: {np.abs(forces).max():.4f} eV/Å")

Testing CHGNet Calculator...
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda

✓ CHGNet Test Passed
  Water Energy: -14.8072 eV
  Max Force: 0.5179 eV/Å


## 7. 후보 데이터베이스 테스트

In [14]:
# 기본 후보 데이터베이스 확인
from src.database.candidates import DEFAULT_CANDIDATES

print(f"Default Candidates: {len(DEFAULT_CANDIDATES)}")
print("\nCandidate List:")
print("-" * 60)

for i, c in enumerate(DEFAULT_CANDIDATES, 1):
    print(f"{i:2}. {c.name:<15} | {c.formula:<20} | CO2↓ {c.co2_reduction}% | Tier {c.tier}")

Default Candidates: 16

Candidate List:
------------------------------------------------------------
 1. BFS             | Al2Ca6MgO16Si4       | CO2↓ 85% | Tier 1
 2. SteelSlag       | Ca8Fe2MgO16Si3       | CO2↓ 75% | Tier 1
 3. EAFSlag         | AlCa5Fe3O16Si4       | CO2↓ 75% | Tier 1
 4. FlyAshF         | Al3CaFeO16Si6        | CO2↓ 85% | Tier 2
 5. FlyAshC         | Al2Ca4O16Si5         | CO2↓ 85% | Tier 2
 6. BottomAsh       | Al3CaFe2O16Si5       | CO2↓ 75% | Tier 2
 7. CopperSlag      | AlCa2Fe5O16Si4       | CO2↓ 75% | Tier 3
 8. RedMud          | Al4Fe4O16Si2Ti       | CO2↓ 65% | Tier 3
 9. NickelSlag      | Fe5Mg2O16Si4         | CO2↓ 75% | Tier 3
10. SilicaFume      | O16Si8               | CO2↓ 88% | Tier 4
11. RiceHuskAsh     | KO16Si8              | CO2↓ 92% | Tier 4
12. POFA            | CaK2O16Si6           | CO2↓ 88% | Tier 4
13. WasteGlass      | Ca2Na2O16Si6         | CO2↓ 75% | Tier 5
14. CeramicWaste    | Al4O16Si6            | CO2↓ 65% | Tier 5
15. Metakaolin   

In [15]:
# 데이터베이스 생성 및 저장 테스트
from src.database.candidates import create_default_database

db_path = src.DATA_DIR / "candidates" / "candidates.json"
db_path.parent.mkdir(parents=True, exist_ok=True)

db = create_default_database(db_path)

print(f"\n✓ Database created: {db_path}")
print(f"  Total candidates: {len(db)}")


✓ Database created: c:\cement_final\data\candidates\candidates.json
  Total candidates: 16


## 8. 폴더 구조 확인

In [16]:
# 프로젝트 폴더 구조 확인
folders = [
    src.PROJECT_ROOT / "src",
    src.CONFIG_DIR,
    src.DATA_DIR,
    src.RESULTS_DIR,
    src.STRUCTURES_DIR,
    src.TRAJECTORIES_DIR,
    src.FIGURES_DIR,
    src.PROJECT_ROOT / "notebooks" / "pipeline",
    src.PROJECT_ROOT / "notebooks" / "archive",
    src.PROJECT_ROOT / "paper",
]

print("Project Structure:")
print("=" * 50)

for folder in folders:
    status = "✓" if folder.exists() else "✗"
    rel_path = folder.relative_to(src.PROJECT_ROOT)
    print(f"  {status} {rel_path}/")

Project Structure:
  ✓ src/
  ✓ config/
  ✓ data/
  ✓ results/
  ✓ structures/
  ✓ trajectories/
  ✓ figures/
  ✓ notebooks\pipeline/
  ✓ notebooks\archive/
  ✓ paper/


## 9. 최종 검증 요약

In [17]:
print("\n" + "=" * 60)
print("SETUP VALIDATION SUMMARY")
print("=" * 60)

checks = [
    ("Python Environment", True),
    ("GPU (CUDA)", torch.cuda.is_available()),
    ("ASE Package", True),
    ("CHGNet Package", True),
    ("src/ Modules", True),
    ("Config Files", True),
    ("Candidate Database", len(db) > 0),
]

all_passed = True
for name, status in checks:
    icon = "✓" if status else "✗"
    print(f"  {icon} {name}")
    if not status:
        all_passed = False

print("\n" + "-" * 60)

if all_passed:
    print("\n✅ ALL CHECKS PASSED - Ready for Pipeline Execution")
    print("\n📋 Next Step: 02_Baseline_Reference.ipynb")
else:
    print("\n❌ SOME CHECKS FAILED - Please fix issues before proceeding")


SETUP VALIDATION SUMMARY
  ✓ Python Environment
  ✓ GPU (CUDA)
  ✓ ASE Package
  ✓ CHGNet Package
  ✓ src/ Modules
  ✓ Config Files
  ✓ Candidate Database

------------------------------------------------------------

✅ ALL CHECKS PASSED - Ready for Pipeline Execution

📋 Next Step: 02_Baseline_Reference.ipynb


---

## 완료

이 노트북에서 확인한 항목:

1. ✅ Python 환경 및 필수 패키지
2. ✅ GPU (CUDA) 가용성
3. ✅ CHGNet 계산기 동작
4. ✅ src/ 모듈 import
5. ✅ 설정 파일 로딩
6. ✅ 후보 데이터베이스

**다음 단계**: `02_Baseline_Reference.ipynb`에서 C3S 기준점을 확립합니다.